In [11]:
import pandas as pd
import numpy as np
import os
import pymysql
import re
import pandas as pd
import re
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to C:\Users\Rafidatus
[nltk_data]     Salsabilah\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Rafidatus
[nltk_data]     Salsabilah\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
def read_mysql_table(table, host='localhost', user='root', password='salsa', database='review'):
    # Establish a connection to the MySQL database
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    
    query = f"SELECT * FROM {table}"
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Convert the result to a Pandas DataFrame
    df = pd.DataFrame(result)
    
    # Assign column names based on the cursor description
    df.columns = [column[0] for column in cursor.description]
    
    # Close the cursor and the database connection
    cursor.close()
    connection.close()
    
    return df

In [29]:
table_name = 'input_review'
df = read_mysql_table(table_name)
df.head()

,id_review,nama,tanggal,review
0,42,Hanin,2024-01-03,Bagus sekali aplikasi ini
1,43,Jojo,2024-01-03,Buruk sekali aplikasi ini
2,44,Kirana,2024-01-03,Aplikasi ini sangat membantu sekali


In [30]:
# membersihkan dan memproses teks pada kolom 'content'

def pre_process(text):
    text = text.lower()
    text = re.sub(r"\d+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = text.strip()
    pisah = text.split()
    tokens = word_tokenize(text)

    factory = StopWordRemoverFactory()
    stopword = factory.create_stop_word_remover()
    text = stopword.remove(text)

    return text

df['content'] = df['review'].apply(pre_process)

print(df.head())


   id_review    nama     tanggal                               review  \
0         42   Hanin  2024-01-03            Bagus sekali aplikasi ini   
1         43    Jojo  2024-01-03            Buruk sekali aplikasi ini   
2         44  Kirana  2024-01-03  Aplikasi ini sangat membantu sekali   

                           content  
0            bagus sekali aplikasi  
1            buruk sekali aplikasi  
2  aplikasi sangat membantu sekali  


In [31]:
review = df['content']

In [32]:
# Specify the file path of the pickle file
file_path = 'C:/Users/Rafidatus Salsabilah/OneDrive/Documents/analisis/reviews.pkl'

# Read the pickle file
with open(file_path, 'rb') as file:
    data_train = pickle.load(file)

In [33]:
data_train

['baru mulai moga bermanfaat sejauh aplikasinya bagus lebih bagus misal detector bar skincare yg digunain jadi pas masukin jenis skincare langsung scan jadi lbh praktis gk usah ngetik manual kalau misal ada mungkin sayanya aja belum tahu hehe sukses selalu',
 'bagus sekali mempermudah mengetahui produk cocok mengatadi masalah kilit mengulas produk kita gunakan cocok sekali digunakan memperbaiki kukit wajah bermasalah aplikasinya mudah simple sangat menarik terimakasih troveskin',
 'hallo trovskin udah pernah download terus baru download ternyata sebanyak manfaatnya karena hadiahredeem point bisa analisis wajah dan tahu produk mana aja yg cocok kulit kita😁 se mantuls love it❤',
 'suka banget sama apps troveskin aplikasi yg me rekomendasikan produk apa yg cocok berdasarkan masalah yg dikulit selain bisa dapet poin kuis yg isi senengnya poinnya bisa ditukar skincare kuy download aplikasinyaa',
 'baru pertama bikin ulasan aplikasi itupun buat dapetin point hehehe tp overall aplikasinya kom

In [34]:
# pembuatan vector kata
vectorizer = TfidfVectorizer()
train_vector = vectorizer.fit_transform(data_train)
reviews2 = ["".join(r) for r in review]

In [35]:
load_model = pickle.load(open('C:/Users/Rafidatus Salsabilah/OneDrive/Documents/analisis/modelsv.pkl','rb'))

result = []

for test in reviews2:
    test_data = [str(test)]
    test_vector = vectorizer.transform(test_data).toarray()
    pred = load_model.predict(test_vector)
    result.append(pred[0])

C:\Users\Rafidatus Salsabilah\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [36]:
result

[5, 5, 5]

In [37]:
from sklearn.utils.multiclass import unique_labels
unique_labels(result)

array([5], dtype=int64)

In [38]:
df['label'] = result

In [42]:
def delete_all_data_from_table(table, host='localhost', user='root', password='salsa', database='review'):
    # Establish a connection to the MySQL database
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    
    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()
    
    # Delete all data from the specified table
    query = f"DELETE FROM {table}"
    cursor.execute(query)
    
    # Commit the changes
    connection.commit()
    
    # Close the cursor and the database connection
    cursor.close()
    connection.close()

In [43]:
delete_all_data_from_table('input_review')

In [44]:
def insert_df_into_hasil_model(df, host='localhost', user='root', password='salsa', database='review'):
    # Establish a connection to the MySQL database
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )

    # Create a cursor object to execute SQL queries
    cursor = connection.cursor()

    # Insert each row from the DataFrame into the 'hasil_model' table
    for index, row in df.iterrows():
        query = "INSERT INTO hasil_model (id_review, nama, tanggal, review, label) VALUES (%s, %s, %s, %s, %s)"
        cursor.execute(query, (row['id_review'], row['nama'], row['tanggal'], row['review'], row['label']))

    # Commit the changes
    connection.commit()

    # Close the cursor and the database connection
    cursor.close()
    connection.close()

In [45]:
insert_df_into_hasil_model(df)

In [46]:
table_name = 'hasil_model'
hasil_df = read_mysql_table(table_name)
hasil_df.to_csv('C:/Users/Rafidatus Salsabilah/OneDrive/Documents/analisis/hasil_modelterbaru.csv')